## Excitability of Network

Construct network such that stimuli recieved from the first state
impacts processing of next state, the memory of the system.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spikey
np.random.seed(0)

In [ ]:
def quincience_time(w_matrix, neuron, **config):
    neuron.reset()

    for i in range(config['processing_time']):
        i_spikes = [0 if i else config['input_magnitude']]
        b_spikes = neuron()

        spikes = np.append(i_spikes, b_spikes)

        neuron += np.sum(w_matrix * spikes.reshape((-1, 1)), axis=0)

        if i and not np.sum(b_spikes):
            return i
        
    return i + 15

In [ ]:
## Static Configuration, multiple runs
N_INPUTS = 1
N_NEURONS = 200
neuron_conf = {
    'n_neurons': N_NEURONS,
    'magnitude': 1,
    'firing_threshold': 2,
    'potential_decay': .05,
    'refractory_period': 0,
}
weight_conf = {
    'n_inputs': N_INPUTS,
    'weight_generator': lambda shape: np.random.uniform(0, 2, size=shape),
    'max_weight': 2,
}
config = {
    'processing_time': 20,
    'input_magnitude': 4,
    **neuron_conf,
    **weight_conf
}

connection_prob = .1

times = []
for _ in range(1000):
    config.update({
        'matrix_mask': np.random.uniform(0, 1, size=(N_INPUTS+N_NEURONS, N_NEURONS)) <= connection_prob
    })

    weight = spikey.weight.Random(**config)
    neuron = spikey.neuron.Neuron(**config)

    t = quincience_time(weight, neuron, **config)

    times.append(t)

sns.distplot(times)
plt.title("Quincience Times per Static Configuration")
plt.xlabel("Time till Quiet")
plt.ylabel("%")
plt.show()

In [ ]:
N_INPUTS = 1
N_NEURONS = 100
neuron_conf = {
    'n_neurons': N_NEURONS,
    'magnitude': 1,
    'firing_threshold': 2,
    'potential_decay': .05,
    'refractory_period': 0,
}
weight_conf = {
    'n_inputs': N_INPUTS,
    'weight_generator': lambda shape: np.random.uniform(0, 2, size=shape),
    'max_weight': 2,
}
config = {
    'processing_time': 20,
    'input_magnitude': 4,
    **neuron_conf,
    **weight_conf
}

PARAM_VALUES = np.linspace(0, .5, 5)

average_times = []
for connection_prob in PARAM_VALUES:
    config.update({
        'matrix_mask': np.random.uniform(0, 1, size=(N_INPUTS+N_NEURONS, N_NEURONS)) <= connection_prob
    })

    times = []
    for _ in range(1000):
        weight = spikey.weight.Random(**config)
        neuron = spikey.neuron.Neuron(**config)
        t = quincience_time(weight, neuron, **config)
        times.append(t)

    average_times.append(np.mean(times))

plt.plot(PARAM_VALUES, average_times)
plt.title(f"Average Quincience Time vs Weight Connection Probability")
plt.xlabel("Connection Probability")
plt.ylabel("Average Quincience Tim{e")
plt.show()